# Implementation of a simple GAN using a 2 layered neural net.

In [10]:
# Importing TensorFlow

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import *
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
# Hyperparameters

batch = 32
X_dim = 784
Z_dim = 64
h_dim = 128
alpha = 0.001

In [12]:
def plot(samples):
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig
    

In [17]:
# Function to initialise weight matrix of a given shape
#def weight_initialise(shape):
#    value = tf.truncated_normal(shape,stddev=0.1)
#    return tf.Variable(value)

# Setting up the Xavier Initialisation Function for the weights

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size,stddev=xavier_stddev)

In [18]:
# Defining variables for the Discriminator net.

X = tf.placeholder(tf.float32, shape = [None,X_dim])

#Layer 1
D_Weights1 = tf.Variable(xavier_init([X_dim,h_dim]))
D_Bias1 = tf.Variable(tf.zeros(shape=[h_dim]))

#Layer 2
D_Weights2 = tf.Variable(xavier_init([h_dim,1]))
D_Bias2 = tf.Variable(tf.zeros(shape=[1]))


theta_D = [D_Weights1, D_Weights2, D_Bias1, D_Bias2]

In [19]:
# Similarly, for the Generator...

z = tf.placeholder(tf.float32,shape=[None,Z_dim])

#Layer 1
G_Weights1 = tf.Variable(xavier_init([Z_dim,h_dim]))
G_Bias1 = tf.Variable(tf.zeros(shape=[h_dim]))

#Layer 2
G_Weights2 = tf.Variable(xavier_init([h_dim,X_dim]))
G_Bias2 = tf.Variable(tf.zeros(shape=[X_dim]))


theta_G = [G_Weights1, G_Weights2, G_Bias1, G_Bias2]

In [20]:
# Defining the generator function

def generator(z):
    G_Activation1 = tf.nn.relu(tf.matmul(z,G_Weights1)+G_Bias1)
    G_Activation2 = tf.matmul(G_Activation1,G_Weights2) + G_Bias2
    generated = tf.nn.sigmoid(G_Activation2)
    return generated


# Defining the discriminator

def discriminator(x):
    D_Activation1 = tf.nn.relu(tf.matmul(x,D_Weights1)+D_Bias1)
    D_Activation2 = tf.matmul(D_Activation1,D_Weights2) + D_Bias2
    return D_Activation2
    

In [21]:
# Function to create initial z samples that are fed to generator

def sample_z(m,n):
    return np.random.uniform(-1., 1., size=[m, n])

In [22]:
G_sample = generator(z)

D_Real  = discriminator(X)

D_Fake  = discriminator(G_sample)

D_Target = 1./ batch

G_Target = 1./ (batch*2)

Z = tf.reduce_sum(tf.exp(-D_Real)) + tf.reduce_sum(tf.exp(-D_Fake))

D_loss = tf.reduce_sum(D_Target * D_Real) + tf.log(Z + 1e-8)
G_loss = tf.reduce_sum(G_Target * D_Real) + tf.reduce_sum(G_Target * D_Fake) + tf.log(Z + 1e-8)



D_solver = (tf.train.AdamOptimizer(learning_rate=alpha)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=alpha)
            .minimize(G_loss, var_list=theta_G))

sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [23]:
# Training
if not os.path.exists('Simple_GAN_Images/'):
    os.makedirs('Simple_GAN_Images/')
i = 0
for it in range(1000000):
    X_mb, _ = mnist.train.next_batch(batch)
    z_mb = sample_z(batch, Z_dim)

    _, D_loss_curr = sess.run(
        [D_solver, D_loss], feed_dict={X: X_mb, z: z_mb}
    )

    _, G_loss_curr = sess.run(
        [G_solver, G_loss], feed_dict={X: X_mb, z: z_mb}
    )

    if it % 1000 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))
    if it % 100000 == 0:
        samples = sess.run(G_sample,feed_dict={z: sample_z(16, Z_dim)})
        
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)),bbox_inches='tight')
        i+=1
        plt.close(fig)

Iter: 0; D_loss: 4.625; G_loss: 4.432
Iter: 1000; D_loss: 3.537; G_loss: 5.519
Iter: 2000; D_loss: 3.658; G_loss: 4.766
Iter: 3000; D_loss: 3.7; G_loss: 4.972
Iter: 4000; D_loss: 3.764; G_loss: 4.78
Iter: 5000; D_loss: 3.715; G_loss: 4.73
Iter: 6000; D_loss: 3.744; G_loss: 4.532
Iter: 7000; D_loss: 3.765; G_loss: 4.594
Iter: 8000; D_loss: 3.712; G_loss: 4.562
Iter: 9000; D_loss: 3.808; G_loss: 4.571
Iter: 10000; D_loss: 3.7; G_loss: 4.711
Iter: 11000; D_loss: 3.866; G_loss: 4.542
Iter: 12000; D_loss: 3.839; G_loss: 4.644
Iter: 13000; D_loss: 3.859; G_loss: 4.524
Iter: 14000; D_loss: 3.838; G_loss: 4.628
Iter: 15000; D_loss: 3.811; G_loss: 4.511
Iter: 16000; D_loss: 3.845; G_loss: 4.458
Iter: 17000; D_loss: 3.834; G_loss: 4.553
Iter: 18000; D_loss: 3.832; G_loss: 4.601
Iter: 19000; D_loss: 3.845; G_loss: 4.554
Iter: 20000; D_loss: 3.941; G_loss: 4.519
Iter: 21000; D_loss: 3.762; G_loss: 4.469
Iter: 22000; D_loss: 3.773; G_loss: 4.532
Iter: 23000; D_loss: 3.841; G_loss: 4.492
Iter: 24000

KeyboardInterrupt: 